# TODO
- automatic creation of the bucket;
- automatic creation of credentials (test root)
- model service
- prefect deployment

In [1]:
# download dataset and store it in s3 bucket
!python ./scripts/download_data.py

In [1]:
# run manual hyperparameter optimization
!python ./scripts/hpo.py 90

2023/08/05 19:27:18 INFO mlflow.tracking.fluent: Experiment with name 'red-wine-quality-hyperopt' does not exist. Creating a new experiment.
[I 2023-08-05 19:27:18,281] A new study created in memory with name: no-name-4bdb5175-49d5-4004-a5cc-7944106477e9
[I 2023-08-05 19:27:20,162] Trial 0 finished with value: 0.645 and parameters: {'n_estimators': 1100, 'max_features': 'sqrt', 'max_depth': 60, 'bootstrap': False, 'min_samples_leaf': 2, 'min_samples_split': 10, 'random_state': 0, 'n_jobs': -1}. Best is trial 0 with value: 0.645.
[I 2023-08-05 19:27:22,589] Trial 1 finished with value: 0.63 and parameters: {'n_estimators': 1950, 'max_features': 'log2', 'max_depth': 60, 'bootstrap': False, 'min_samples_leaf': 1, 'min_samples_split': 2, 'random_state': 0, 'n_jobs': -1}. Best is trial 0 with value: 0.645.
[I 2023-08-05 19:27:22,758] Trial 2 finished with value: 0.66 and parameters: {'n_estimators': 50, 'max_features': 'sqrt', 'max_depth': 90, 'bootstrap': True, 'min_samples_leaf': 2, 'min_

In [2]:
# manually register the best random forest model via prefect flow
!python ./scripts/register_best_model.py

2023/08/05 19:29:32 INFO mlflow.tracking.fluent: Experiment with name 'red-wine-quality-production' does not exist. Creating a new experiment.
19:29:32.914 | INFO    | prefect.engine - Created flow run 'quaint-vole' for flow 'Main Registration Flow'
19:29:32.915 | INFO    | Flow run 'quaint-vole' - View at http://prefect-server:4200/flow-runs/flow-run/f4b71da4-3f01-4b13-b93d-9bde0f662747
19:29:33.059 | INFO    | Flow run 'quaint-vole' - Created task run 'Prepare Data-0' for task 'Prepare Data'
19:29:33.060 | INFO    | Flow run 'quaint-vole' - Executing 'Prepare Data-0' immediately...
19:29:33.190 | INFO    | Task run 'Prepare Data-0' - Finished in state Completed()
19:29:33.203 | INFO    | Flow run 'quaint-vole' - Created task run 'Get Parameters-0' for task 'Get Parameters'
19:29:33.204 | INFO    | Flow run 'quaint-vole' - Executing 'Get Parameters-0' immediately...
19:29:33.273 | INFO    | Task run 'Get Parameters-0' - Finished in state Completed()
19:29:33.289 | INFO    | Flow run '

In [1]:
# register minio block
!python ./scripts/register_minio_block.py

In [7]:
# register & run a new prefect flow deployment
# you can adjust the cron parameter for the prefered schedule
!prefect deployment build ./scripts/register_best_model.py:main_flow -n rfc-red-wine -p default-agent-pool -q red-wine -sb remote-file-system/minio --cron "0 0 * * *" -o ./scripts/main_flow_deployment.yaml
!prefect deployment apply ./scripts/main_flow_deployment.yaml
!prefect deployment run "Main Registration Flow/rfc-red-wine"

Found flow 'Main Registration Flow'
Deployment YAML created at 
'/home/jovyan/work/scripts/main_flow_deployment.yaml'.
Successfully uploaded 2 files to s3://mlflow-artifacts/
Successfully loaded 'rfc-red-wine'
Deployment 'Main Registration Flow/rfc-red-wine' successfully created with id 
'c74f568b-a3f7-4e8e-a26a-397852014a71'.
View Deployment in UI: 
http://prefect-server:4200/deployments/deployment/c74f568b-a3f7-4e8e-a26a-397852
014a71

To execute flow runs from this deployment, start an agent that pulls work from 
the 'default-agent-pool' work pool:
$ prefect agent start -p 'default-agent-pool'
Creating flow run for deployment 'Main Registration Flow/rfc-red-wine'...
Created flow run 'statuesque-tuna'.
└── UUID: 43e6c38d-aa2b-4d9a-98c4-e7a255d8147f
└── Parameters: {}
└── Scheduled start time: 2023-08-07 20:09:07 UTC (now)
└── URL: 
http://prefect-server:4200/flow-runs/flow-run/43e6c38d-aa2b-4d9a-98c4-e7a255d814
7f
